<a href="https://colab.research.google.com/github/fluffybird2323/Sentiment-analysis-on-Amazon-reviews-/blob/master/Sentiment_Analysis_on_Amazon_Reviews%20with%20Convnet%20and%20GloVe%20Encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer(num_words=10000)

with open('Book1.csv',encoding = 'ISO-8859-1') as fin:
    for line in fin:      
        t.fit_on_texts( line.split()) # Fitting the tokenizer line-by-line.

M = []

with open('Book1.csv',encoding = 'ISO-8859-1') as fin:
    for line in fin:
        # Converting the lines into matrix, line-by-line.
        m = t.texts_to_sequences([line])[0]
        M.append(m)

Using TensorFlow backend.


In [0]:
import numpy as np

In [0]:
X = np.array(M)
train_size = int(len(X) * .75)
X = X[:7134]

In [4]:
X.shape

(7134,)

In [5]:
from keras.preprocessing.sequence import pad_sequences
word_index = t.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(X, maxlen=1000)

Found 11975 unique tokens.


In [6]:
embeddings_index = {}
f = open('glove.6B.50d.txt', encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
EMBEDDING_DIM = 50
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding
MAX_SEQUENCE_LENGTH = 1000
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [9]:
import csv
with open('Y.csv') as raw:
    stars = np.array(list(csv.reader(raw,delimiter=',')))
print(stars.reshape(7134,1))    

[['4']
 ['5']
 ['5']
 ...
 ['5']
 ['5']
 ['5']]


In [0]:
Y = stars 
from sklearn.utils import shuffle
data,Y = shuffle(data,Y)
Xtrain = data[:train_size]
Xtest = data[train_size:]
Ytrain = Y[:train_size] 
Ytest = Y[train_size:]

In [0]:
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
Ytrain = to_categorical(Ytrain.astype(np.int) -1) 
Ytest = to_categorical(Ytest.astype(np.int) -1) 

In [12]:
from keras import regularizers
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.models import Model

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(512, 5, activation='relu',kernel_regularizer=regularizers.l2(0.009))(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Flatten()(x)
x = Dense(1024, activation='elu',kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(rate = 0.1)(x)
x = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.002))(x)
preds = Dense(len(Ytrain.T), activation='softmax')(x)


model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print(model.metrics_names)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
['loss', 'acc']


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          598800    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 512)          128512    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 512)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 101888)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              104334336 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [0]:
batch_size = 1024
epochs = 100

In [17]:
history = model.fit(Xtrain, Ytrain, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)
score = model.evaluate(Xtest, Ytest, batch_size=batch_size, verbose=1)
score1 = model.evaluate(Xtrain, Ytrain, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Train loss:', score1[0])
print('Train accuracy:', score1[1])

Train on 4893 samples, validate on 544 samples
Epoch 1/100
4893/4893 [==============================] - 5s 924us/step - loss: 1.6851 - acc: 0.7996 - val_loss: 1.5307 - val_acc: 0.8004
Epoch 2/100
4893/4893 [==============================] - 3s 561us/step - loss: 1.4339 - acc: 0.8008 - val_loss: 1.2920 - val_acc: 0.8004
Epoch 3/100
4893/4893 [==============================] - 3s 562us/step - loss: 1.2115 - acc: 0.8149 - val_loss: 1.1007 - val_acc: 0.8478
Epoch 4/100
4893/4893 [==============================] - 3s 568us/step - loss: 1.0519 - acc: 0.8565 - val_loss: 0.9942 - val_acc: 0.8515
Epoch 5/100
4893/4893 [==============================] - 3s 569us/step - loss: 0.9477 - acc: 0.8628 - val_loss: 0.8998 - val_acc: 0.8603
Epoch 6/100
4893/4893 [==============================] - 3s 569us/step - loss: 0.8637 - acc: 0.8658 - val_loss: 0.8334 - val_acc: 0.8585
Epoch 7/100
4893/4893 [==============================] - 3s 574us/step - loss: 0.8014 - acc: 0.8661 - val_loss: 0.7807 - val_acc: 0

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [134]:
score = model.evaluate(Xtest, Ytest, batch_size=batch_size, verbose=1)
score1 = model.evaluate(Xtrain, Ytrain, batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Train loss:', score1[0])
print('Train accuracy:', score1[1])

5437/5437 [==============================] - 1s 253us/step
Test loss: 0.5473201404828919
Test accuracy: 0.8588096767786888
Train loss: 0.3073698836739326
Train accuracy: 0.9494574304141826


In [0]:
model.save('SA.h5')

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')